## Visualizing Materialized Annotations in Neuroglancer

In [33]:
from nglui.statebuilder import ImageLayerConfig, PointMapper, AnnotationLayerConfig, StateBuilder
from caveclient import CAVEclient

In [34]:
url = "https://global.brain-wire-test.org/"
datastack = "h01_c3_flat"
client = CAVEclient(datastack_name=datastack, server_address=url)

### Get Annotations from Materialized Table

In [35]:
annotationsDf = client.materialize.query_table(table="cells",
                                              desired_resolution=[4,4,40]
                                              )

> https://caveclient.readthedocs.io/en/latest/guide/materialization.html#desired-resolution
If you want positions in 4,4,40nm voxel coordinates to use with cloud-volume or neuroglancer you would pass desired_resolution=[4,4,40].

In [36]:
annotationsDf.tail()

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
49372,49373,2023-06-15 19:29:36.516592+00:00,NaN,t,"0, 0, 0",ASTROCYTE,104452849731108865,864691132482582976,"[471970.0, 382770.0, 1584.825]"
49373,49374,2023-06-15 19:29:36.517171+00:00,NaN,t,"0, 0, 0",ASTROCYTE,100224335913943043,864691132258960726,"[410608.0, 286964.0, 4330.425]"
49374,49375,2023-06-15 19:29:36.517764+00:00,NaN,t,"16, 0, 0",UNCLASSIFIEDNEURON,115930719393415168,864691132393049686,"[638326.0, 498068.0, 108.075]"
49375,49376,2023-06-15 19:29:36.518349+00:00,NaN,t,"0, 0, 0",MG_OPC,108808359288766465,864691132207193954,"[534924.0, 273004.0, 2220.075]"
49376,49377,2023-06-15 19:29:36.518931+00:00,NaN,t,"0, 0, 0",INTERNEURON,125761110870589440,864691132401212067,"[782030.0, 181592.0, 2724.975]"


### Create neuroglancer state

In [37]:
img_source = client.info.image_source()
img_layer = ImageLayerConfig(name='h01', source=img_source)

In [38]:
points = PointMapper(point_column="pt_position")
anno_layer = AnnotationLayerConfig(name="annos", mapping_rules=[points])

In [39]:
sb = StateBuilder([img_layer, anno_layer], url_prefix='https://ngl.brain-wire.org/')
d = sb.render_state(annotationsDf[1000:1500], 
                    return_as='dict', 
                    compatibility_mode=True, 
                    url_prefix='https://ngl.brain-wire.org/')

In [40]:
# set dimensions for annotation layer
d['layers'][1]['source'][0]['transform']['outputDimensions']={
            "x": [
              4e-9,
              "m"
            ],
            "y": [
              4e-9,
              "m"
            ],
            "z": [
              3.3e-8,
              "m"
            ]
          }

In [41]:
d['crossSectionScale']=7.5e-8 # -> it is weird. 7.5e-8 is 7.5um, not 0.075um.

In [42]:
d

OrderedDict([('selectedLayer',
              OrderedDict([('layer', 'annos'), ('visible', True)])),
             ('showSlices', False),
             ('layout', 'xy-3d'),
             ('crossSectionScale', 7.5e-08),
             ('projectionScale', 2000.0),
             ('layers',
              [OrderedDict([('type', 'image'),
                            ('source',
                             [{'url': 'precomputed://gs://h01-release/data/20210601/4nm_raw'}]),
                            ('name', 'h01')]),
               OrderedDict([('type', 'annotation'),
                            ('source',
                             [OrderedDict([('url', 'local://annotations'),
                                           ('transform',
                                            OrderedDict([('outputDimensions',
                                                          {'x': [4e-09, 'm'],
                                                           'y': [4e-09, 'm'],
                                

In [43]:
state_id=client.state.upload_state_json(d)

In [44]:
url=client.state.build_neuroglancer_url(state_id, 
                                        ngl_url = 'http://ngl.brain-wire.org/').replace("/?json_url=", "#!middleauth+")
url

'http://ngl.brain-wire.org#!middleauth+https://global.brain-wire-test.org//nglstate/api/v1/5718286331281408'